> Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved BSD-3 license.<br>
> Copyright (c) 2021, Felipe N. Schuch.<br>
> [@fschuch](https://twitter.com/fschuch)

In [1]:
import tools

In [2]:
import os.path
import xcompact3d_toolbox as x3d
import numpy as np
from tqdm.notebook import tqdm as tqdm

In [3]:
x3d.param["mytype"] = np.float32

In [4]:
cases = [f"{s}-{us}" for s in "1.25 2.5 5.0 10.0".split() for us in "0 15 30".split()]

## Layer-averaged quantities

The complete spatio-temporal analysis of the relevant quantities is possible in a layer averaged context per width unit, that is computed according to the equations

$$
Uh(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} u_1(x_1,x_2,x_3,t) ~ dx_2 dx_3
$$
$$
U^2h(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} \big( u_1(x_1,x_2,x_3,t) \big)^2 ~ dx_2 dx_3
$$
$$
UCh(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} u_1 (x_1,x_2,x_3,t) ~ c_t (x_1,x_2,x_3,t) ~ dx_2 dx_3
$$

For the vertical integration, $x_{2r}$ represents the bed position and $x_{2i}$ represents the interface between the underflow turbidity current and the ambient fluid, considered in this work as the position where $u_1 c_t = 0.005$.
Then, they can be used to compute layer-averaged velocity $U = U^2h/Uh$, flow depth $H = (Uh)^2/U^2h$, flow discharge $Q = Uh$, concentration $C = UCh/Uh$ and local densimetric Froude number $Fr$ (see [01-Computing-and-Plotting.ipynb](01-Computing-and-Plotting.ipynb)).

In [5]:
def layer_averaged(dataset):
    #
    dataset_chunked = dataset.x3d.pencil_decomp("y")
    #
    uc = (dataset_chunked.ux * dataset_chunked.phi).compute()
    #
    mask = ~((uc < 0.005) & (dataset_chunked.phi < 0.75))
    #
    dataset["Uh"] = -(dataset_chunked.ux).where(mask, 0.0).integrate("y").compute()
    dataset["U2h"] = -(
        (dataset_chunked.ux ** 2.0).where(mask, 0.0).integrate("y").compute()
    )
    dataset["UCh"] = -(uc).where(mask, 0.0).integrate("y").compute()
    #
    dataset.Uh.attrs = {"name": "Layer-averaged Uh", "long_name": r"$Uh$"}
    dataset.U2h.attrs = {"name": "Layer-averaged U2h", "long_name": r"$U^2h$"}
    dataset.UCh.attrs = {"name": "Layer-averaged UCh", "long_name": r"$UCh$"}

**Note:** Specifically for the plunging flow configuration, the vertical coordinate was moved to the top of the computational domain, and it points down, so the signals for `Uh`, `U2h` and `UCh` has been inversed.

# Code

The block bellow computes and writes to the disc a data set containing the layer-averaged quantities, in addition to spanwise-averaged bed shear velocity and spanwise-averaged deposition rate.

In [6]:
for case in tqdm(cases, desc="Case"):
    path = tools.get_datapath(case)

    prm = tools.PlumesParameters(loadfile=path)

    coords = prm.get_mesh
    del coords["z"]

    ds = prm.read_all_fields(
        filename_pattern=os.path.join(prm.datapath, "xy-plane", "phi1????"),
        name="phi1",
        steep="iprocessing",
        coords=coords,
        attrs=dict(name="Concentration Field", long_name=r"$c$"),
    ).to_dataset(name="phi")

    for i, var in enumerate("ux uy".split()):
        ds[var] = prm.read_all_fields(
            filename_pattern=os.path.join(prm.datapath, "xy-plane", f"{var}????"),
            name=var,
            steep="iprocessing",
            coords=coords,
            attrs=dict(
                name=f"{tools.VelocityDescription.get(i,'')} Velocity",
                long_name=fr"$u_{i+1}$",
            ),
        )

    coords = prm.get_mesh
    del coords["y"]

    ds["utau"] = prm.read_all_fields(
        filename_pattern=os.path.join(prm.datapath, "xz-plane", "utau????"),
        name=var,
        steep="iprocessing",
        coords=coords,
        attrs=dict(name="Bed shear velocity", long_name=r"$u_\tau$",),
    ).mean("z")

    if prm.uset[0] > 0.0:
        ds["dep"] = prm.read_all_fields(
            filename_pattern=os.path.join(prm.datapath, "xz-plane", "dep1????"),
            name=var,
            steep="iprocessing",
            coords=coords,
        ).mean("z")
    else:
        # Just to get the same shape, deposition is zero whem settling velocity is zero
        ds["dep"] = 0.0 * ds["utau"]
    ds.dep.attrs["name"] = "Deposition rate"
    ds.dep.attrs["long_name"] = r"$\dot{D}$"
        

    # Preparing dataset includes cut to Test Section and add some key attributes
    ds = tools.prepare_dataset(prm, ds)

    ds_steady_state = ds.sel(t=slice(ds.t[-1] - 2000, None)).mean("t", keep_attrs=True)
    layer_averaged(ds_steady_state)
    ds_steady_state.to_netcdf(f"steady-state-case-{case}.nc")

    ds.isel(t=slice(None, None, 2)).to_netcdf(f"xy-planes-case-{case}.nc")

    layer_averaged(ds)
    ds.drop("ux uy phi bed_position".split()).to_netcdf(f"LA-case-{case}.nc")

Case:   0%|          | 0/12 [00:00<?, ?it/s]

This block converts the raw binary containing the 3D fields to NefCDF.

In [7]:
for case in tqdm(cases, desc="Case"):
    path = tools.get_datapath(case)

    prm = tools.PlumesParameters(loadfile=path)

    coords = prm.get_mesh

    ds = prm.read_all_fields(
        filename_pattern=os.path.join(prm.datapath, "3d-snapshot", "phi1????"),
        name="phi1",
        steep="ioutput",
        coords=coords,
        attrs=dict(name="Concentration Field", long_name=r"$c$"),
    ).to_dataset(name="phi")

    for i, var in enumerate("ux uy uz".split()):
        ds[var] = prm.read_all_fields(
            filename_pattern=os.path.join(prm.datapath, "3d-snapshot", f"{var}????"),
            name=var,
            steep="ioutput",
            coords=coords,
            attrs=dict(
                name=f"{tools.VelocityDescription.get(i,'')} Velocity",
                long_name=fr"$u_{i+1}$",
            ),
        )

    # Preparing dataset includes cut to Test Section and add some key attributes
    ds = tools.prepare_dataset(prm, ds)

    if case.startswith("10.0"):
        ds = ds.assign_coords(
            t=[
                250.0,
                500.0,
                750.0,
                1000.0,
                1250.0,
                1500.0,
                1750.0,
                2000.0,
                3000.0,
                4000.0,
                5000.0,
                6000.0,
            ]
        )
    else:
        ds = ds.assign_coords(
            t=[
                250.0,
                500.0,
                750.0,
                1000.0,
                1250.0,
                1500.0,
                1750.0,
                2000.0,
                3000.0,
                4000.0,
            ]
        )
    #clear bellow channel's bed and save file
    ds.where(ds.y < ds.bed_position, 0.0).to_netcdf(f"3d-case-{case}.nc")

Case:   0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
ds

<xarray.Dataset>
Dimensions:       (t: 12, x: 801, y: 241, z: 32)
Coordinates:
  * x             (x) float32 0.0 0.3125 0.625 0.9375 ... 249.375 249.6875 250.0
  * y             (y) float32 33.75 33.609375 33.46875 ... 0.28125 0.140625 0.0
  * z             (z) float32 0.0 0.25 0.5 0.75 1.0 ... 6.75 7.0 7.25 7.5 7.75
  * t             (t) float64 250.0 500.0 750.0 1e+03 ... 4e+03 5e+03 6e+03
Data variables:
    phi           (t, x, y, z) float32 4.692858e-06 ... 1.17108335e-26
    ux            (t, x, y, z) float32 -2.8595627e-05 ... 0.0047625257
    uy            (t, x, y, z) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    uz            (t, x, y, z) float32 2.6460768e-08 ... -1.8506832e-08
    uset          float64 0.003
    Ri0           float64 0.001891
    Fr0           float64 23.0
    Re            float64 2.5e+03
    S             float64 0.1
    bed_position  (x) float32 1.0 1.03125 1.0625 ... 25.9375 25.96875 26.0
Attributes:
    title:    Plunging condition for particle-laden flows over sloping bottom...
    authors:  F. N. Schuch, E. Meiburg & J. H. Silvestrini
    url:      https://github.com/fschuch/incompact3d_plunging_criterion
    doi:      10.5281/zenodo.4044388
    license:  Creative Commons Attribution 4.0 International